# Введение в MapReduce модель на Python


In [48]:
# requires python 3.6+
from typing import NamedTuple
from typing import Iterator

In [49]:
def MAP(_, row: NamedTuple):
    if row.gender == "female":
        yield (row.age, row)


def REDUCE(age: str, rows: Iterator[NamedTuple]):
    summ = 0
    count = 0
    for row in rows:
        summ += row.social_contacts
        count += 1
    if count > 0:
        yield (age, summ / count)
    else:
        yield (age, 0)

Модель элемента данных

In [50]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str

In [51]:
input_collection = [
    User(id=0, age=55, gender="male", social_contacts=20),
    User(id=1, age=25, gender="female", social_contacts=240),
    User(id=2, age=25, gender="female", social_contacts=500),
    User(id=3, age=33, gender="female", social_contacts=800),
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [52]:
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

In [53]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [54]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

In [55]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output)  # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [56]:
def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [57]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [58]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [59]:
list(
    flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )
)

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [60]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [61]:
# requires python 3.6+
from typing import NamedTuple
from typing import Iterator


class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str


input_collection = [
    User(id=0, age=55, gender="male", social_contacts=20),
    User(id=1, age=25, gender="female", social_contacts=240),
    User(id=2, age=25, gender="female", social_contacts=500),
    User(id=3, age=33, gender="female", social_contacts=800),
]


def MAP(_, row: NamedTuple):
    if row.gender == "female":
        yield (row.age, row)


def REDUCE(age: str, rows: Iterator[NamedTuple]):
    summ = 0
    count = 0
    for row in rows:
        summ += row.social_contacts
        count += 1
    if count > 0:
        yield (age, summ / count)
    else:
        yield (age, 0)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [62]:
from typing import Iterator
import numpy as np

mat = np.ones((5, 4))
vec = np.random.rand(4)  # in-memory vector in all map tasks


def MAP(coordinates: (int, int), value: int):
    i, j = coordinates
    yield (i, value * vec[j])


def REDUCE(i: int, products: Iterator[NamedTuple]):
    summ = 0
    for p in products:
        summ += p
    yield (i, summ)


def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i, j])


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.203309181868484),
 (1, 2.203309181868484),
 (2, 2.203309181868484),
 (3, 2.203309181868484),
 (4, 2.203309181868484)]

## Inverted index 

In [63]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]


def RECORDREADER():
    for docid, document in enumerate(documents):
        yield ("{}".format(docid), document)


def MAP(docId: str, body: str):
    for word in set(body.split(" ")):
        yield (word, docId)


def REDUCE(word: str, docIds: Iterator[str]):
    yield (word, sorted(docIds))


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [64]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]


def RECORDREADER():
    for docid, document in enumerate(documents):
        for lineid, line in enumerate(document.split("\n")):
            yield ("{}:{}".format(docid, lineid), line)


def MAP(docId: str, line: str):
    for word in line.split(" "):
        yield (word, 1)


def REDUCE(word: str, counts: Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [65]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None
):
    map_partitions = map(
        lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)),
        INPUTFORMAT(),
    )
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(
                map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))
            ),
            map_partitions,
        )
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(
        lambda reduce_partition: (
            reduce_partition[0],
            flatten(
                map(
                    lambda reduce_input_group: REDUCE(*reduce_input_group),
                    reduce_partition[1],
                )
            ),
        ),
        reduce_partitions,
    )

    print(
        "{} key-value pairs were sent over a network.".format(
            sum(
                [
                    len(vs)
                    for (k, vs) in flatten(
                        [partition for (partition_id, partition) in reduce_partitions]
                    )
                ]
            )
        )
    )
    return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [66]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for docid, document in enumerate(split):
            for lineid, line in enumerate(document.split("\n")):
                yield ("{}:{}".format(docid, lineid), line)

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i : i + split_size])


def MAP(docId: str, line: str):
    for word in line.split(" "):
        yield (word, 1)


def REDUCE(word: str, counts: Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('it', 18)]),
 (1, [('a', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [67]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for value in split:
            yield (value, None)

    split_size = int(np.ceil(len(input_values) / maps))
    for i in range(0, len(input_values), split_size):
        yield RECORDREADER(input_values[i : i + split_size])


def MAP(value: int, _):
    yield (value, None)


def PARTITIONER(key):
    global reducers
    global max_value
    global min_value
    bucket_size = (max_value - min_value) / reducers
    bucket_id = 0
    while (key > (bucket_id + 1) * bucket_size) and (
        (bucket_id + 1) * bucket_size < max_value
    ):
        bucket_id += 1
    return bucket_id


def REDUCE(value: int, _):
    yield (None, value)


partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.03316849979390424),
   (None, 0.044412141008436934),
   (None, 0.0766630856403232),
   (None, 0.10807385783209034),
   (None, 0.10873255505806656),
   (None, 0.12482595203081259),
   (None, 0.15272569872255937),
   (None, 0.24050164770403393),
   (None, 0.25888544703994854),
   (None, 0.2666122806988731),
   (None, 0.2688570723067958),
   (None, 0.296887729957106),
   (None, 0.3210779788027708),
   (None, 0.3783559005883189),
   (None, 0.39238437717656505),
   (None, 0.4703255907721522)]),
 (1,
  [(None, 0.5901782747441229),
   (None, 0.6245135430589264),
   (None, 0.6369427760872235),
   (None, 0.6739026190498255),
   (None, 0.6802374868126899),
   (None, 0.6876747128563389),
   (None, 0.7369049150990232),
   (None, 0.7795729874131213),
   (None, 0.804234078707242),
   (None, 0.8850452993495883),
   (None, 0.8913014933929179),
   (None, 0.9574929941575627),
   (None, 0.988825387726906),
   (None, 0.9937232689818417)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

In [68]:
# requires python 3.6+
import numpy as np
from typing import NamedTuple
from typing import Iterator

Описанные ранее функции, составляющие MapReduce

In [69]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )


class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str


input_collection = [
    User(id=0, age=55, gender="male", social_contacts=20),
    User(id=1, age=25, gender="female", social_contacts=240),
    User(id=2, age=25, gender="female", social_contacts=500),
    User(id=3, age=33, gender="female", social_contacts=800),
]

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

Алгоритм находит пользователя с максимальным числом social_contacts

In [70]:
# Одинаковый ключ группировки (0) для всех пользователей
def MAP_MAX(_, row: NamedTuple):
    yield (0, row)


# Поиск пользователя с максимальным числом social_contacts
def REDUCE_MAX(_, rows: Iterator[NamedTuple]):
    max_contacts = 0
    max_contacts_user = None
    for row in rows:
        if row.social_contacts > max_contacts:
            max_contacts = row.social_contacts
            max_contacts_user = row
    yield max_contacts_user


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output

[User(id=3, age=33, social_contacts=800, gender='female')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


Алгоритм вычисляет среднее значение параметра social_contacts среди всех пользователей

In [71]:
# Одинаковый ключ группировки (0) для всех пользователей
def MAP_AVG(_, row: NamedTuple):
    yield (0, row)


# Подсчет среднего значения social_contacts
def REDUCE_AVG(_, rows: Iterator[NamedTuple]):
    summ = 0
    count = 0
    for row in rows:
        summ += row.social_contacts
        count += 1
    if count > 0:
        yield summ / count
    else:
        yield 0


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
output

[390.0]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [72]:
def groupbykey(iterable):
    # Сортировка по ключу
    iterable = sorted(iterable, key=lambda e: e[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

Проверка на предыдущих примерах

In [73]:
output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
print(f"max: {output}")

output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
print(f"average: {output}")

max: [User(id=3, age=33, social_contacts=800, gender='female')]
average: [390.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

Дубликатами будем считать пользователей с одинаковым id

In [74]:
input_collection = [
    User(id=0, age=54, gender="male", social_contacts=20),
    User(id=0, age=25, gender="female", social_contacts=240),
    User(id=1, age=27, gender="male", social_contacts=642),
    User(id=2, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=247),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=3, age=32, gender="female", social_contacts=753),
]

Представленный ранее код для распределенной процедуры MapReduceDistributed

In [75]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(
    INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None
):
    map_partitions = map(
        lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)),
        INPUTFORMAT(),
    )
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(
                map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))
            ),
            map_partitions,
        )
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(
        lambda reduce_partition: (
            reduce_partition[0],
            flatten(
                map(
                    lambda reduce_input_group: REDUCE(*reduce_input_group),
                    reduce_partition[1],
                )
            ),
        ),
        reduce_partitions,
    )

    print(
        "{} key-value pairs were sent over a network.".format(
            sum(
                [
                    len(vs)
                    for (k, vs) in flatten(
                        [partition for (partition_id, partition) in reduce_partitions]
                    )
                ]
            )
        )
    )
    return reduce_outputs

In [76]:
maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    # Разделение массива пользователей между вычислительными узлами
    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])


# Группировка пользователей по id
def MAP_UNIQUE(_, user: str):
    yield (user.id, user)


# Из множества пользователей с одинаковым id остается только первый
def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output

7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=54, social_contacts=20, gender='male'),
   User(id=2, age=16, social_contacts=123, gender='male')]),
 (1,
  [User(id=1, age=27, social_contacts=642, gender='male'),
   User(id=3, age=31, social_contacts=521, gender='male')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [77]:
# Предикатом C в данном случае является проверка параметра gender
def MAP_SELECT(_, row: NamedTuple):
    if row.gender == "female":
        yield (row, row)


def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=0, age=25, social_contacts=240, gender='female'),
  [User(id=0, age=25, social_contacts=240, gender='female')]),
 (User(id=2, age=35, social_contacts=247, gender='female'),
  [User(id=2, age=35, social_contacts=247, gender='female')]),
 (User(id=3, age=32, social_contacts=753, gender='female'),
  [User(id=3, age=32, social_contacts=753, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

Исключаем из кортежа пользователя параметр gender, если его значение не принадлежит множеству S = ["male", "female"]

In [78]:
input_collection_2 = [
    dict(id=0, age=54, gender="male", social_contacts=20),
    dict(id=0, age=25, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="undefined", social_contacts=642),
    dict(id=2, age=16, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="undefined", social_contacts=247),
    dict(id=3, age=31, gender="male", social_contacts=521),
    dict(id=3, age=32, gender="female", social_contacts=753),
]

Код MapReduce для наглядности

In [79]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

In [80]:
# Группировка по id
def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        # Удаляем поле gender у полученого словаря
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)


def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u["id"], u) for u in input_collection_2]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[(0,
  [{'id': 0, 'age': 54, 'gender': 'male', 'social_contacts': 20},
   {'id': 0, 'age': 25, 'gender': 'female', 'social_contacts': 240}]),
 (1, [{'id': 1, 'age': 27, 'social_contacts': 642}]),
 (2,
  [{'id': 2, 'age': 16, 'gender': 'male', 'social_contacts': 123},
   {'id': 2, 'age': 35, 'social_contacts': 247}]),
 (3,
  [{'id': 3, 'age': 31, 'gender': 'male', 'social_contacts': 521},
   {'id': 3, 'age': 32, 'gender': 'female', 'social_contacts': 753}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значения. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [81]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=6, age=31, gender="male", social_contacts=521),
    User(id=7, age=32, gender="female", social_contacts=753),
]

In [82]:
# Группировка пользователей по id
def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)


# Выходное значение равно (t, t) независимо от присутствия пользователя в только одной или обоих выборках
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=1, age=54, social_contacts=20, gender='male'),
  User(id=1, age=54, social_contacts=20, gender='male')),
 (User(id=2, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=27, social_contacts=642, gender='male'),
  User(id=3, age=27, social_contacts=642, gender='male')),
 (User(id=4, age=16, social_contacts=123, gender='male'),
  User(id=4, age=16, social_contacts=123, gender='male')),
 (User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')),
 (User(id=6, age=31, social_contacts=521, gender='male'),
  User(id=6, age=31, social_contacts=521, gender='male')),
 (User(id=7, age=32, social_contacts=753, gender='female'),
  User(id=7, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [83]:
# Группировка по пользователям
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)


# Возвращает пользователя, только если он присутствует в обоих выборках
def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [84]:
# Группировка по пользователям
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)


# Возвращает пользователей, входящих только в первую выборку (и не входящих во вторую)
def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)


# На первом месте стоит номер выборки
def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]


output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=1, age=54, social_contacts=20, gender='male'),
 User(id=2, age=25, social_contacts=240, gender='female'),
 User(id=3, age=27, social_contacts=642, gender='male'),
 User(id=4, age=16, social_contacts=123, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

Пользователям был добавлен параметр city_id и была создана сущность город (City)

In [85]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    city_id: int


users_collection = [
    User(id=1, age=54, gender="male", social_contacts=20, city_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, city_id=2),
    User(id=3, age=27, gender="male", social_contacts=642, city_id=2),
    User(id=4, age=16, gender="male", social_contacts=123, city_id=3),
    User(id=5, age=35, gender="female", social_contacts=247, city_id=4),
]


class City(NamedTuple):
    id: int
    name: str


cities_collection = [
    City(id=1, name="Samara"),
    City(id=2, name="Moscow"),
    City(id=3, name="Omsk"),
    City(id=4, name="Tomks"),
    City(id=5, name="Belgorod"),
]

In [86]:
# Группировка пользователей по id города
def MAP_JOIN(city_id, row):
    yield (city_id, row)


def REDUCE_JOIN(city_id, rows):
    users = []
    city = None

    # Цикл для определения города (он будет единственным, так как группировка происходит по id города)
    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            city = row

    for row in rows:
        if type(row) is User:
            # Возвращает тройки (a,b,c), где a - пользователь, b - id города, c - сам город
            yield (row, row.city_id, city)


# На первом месте стоит id города, на втором - сам город или пользователь
def RECORDREADER():
    return [(user.city_id, user) for user in users_collection] + [
        (city.id, city) for city in cities_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=54, social_contacts=20, gender='male', city_id=1),
  1,
  City(id=1, name='Samara')),
 (User(id=2, age=25, social_contacts=240, gender='female', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=3, age=27, social_contacts=642, gender='male', city_id=2),
  2,
  City(id=2, name='Moscow')),
 (User(id=4, age=16, social_contacts=123, gender='male', city_id=3),
  3,
  City(id=3, name='Omsk')),
 (User(id=5, age=35, social_contacts=247, gender='female', city_id=4),
  4,
  City(id=4, name='Tomks'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [87]:
# Группировка по id города
def MAP_GROUP(city_id, user, city):
    yield (city_id, user)


# Подсчет количества пользователей в каждом из городов
def REDUCE_GROUP(city_id, rows):
    yield f"city with id={city_id} has {len(rows)} user(s)"


def RECORDREADER():
    return [(city_id, user, city) for user, city_id, city in joined_data]


output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['city with id=1 has 1 user(s)',
 'city with id=2 has 2 user(s)',
 'city with id=3 has 1 user(s)',
 'city with id=4 has 1 user(s)']

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





Представленный ранее код MapReduce для наглядности

In [88]:
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [89]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(
    I, J
)  # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J, K)


# Генерирует элементы второй матрицы вместе с их индексами
def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])


# Использует сгенерированные элементы второй матрицы и возвращает
# их произведения на соответствующие элементы первой матрицы.
# Для каждого элемента N[j,k] генерируется J произведений на M[i,j]
def MAP(k1, v1):
    (j, k) = k1
    w = v1

    # solution code that yield(k2,v2) pairs
    for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)


# Суммирует полученные произведения для вычисления элементов результирующей матрицы
def REDUCE(key, values):
    (i, k) = key

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)

Проверьте своё решение

In [90]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

In [91]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i, k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [92]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)


# Генерирует пары элементов из обоих матриц вместе с соответствующими им индексами
# Элементы в паре предназначены для перемножения и дальнейшего суммирования (M[i,j] и N[i,k])
def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))


# Перемножает полученные элементы двух матриц
# Группирует произведения по индексу, соответствующему клетке в результирующей матрице
# Таким образом, для каждого ключа (i, k) генерируется J произведений
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


# Суммирует полученные произведения для вычисления элементов результирующей матрицы
def REDUCE(key, values):
    (i, k) = key

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)

Проверка решения

In [93]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [94]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    # Так же, как и раньше, генерируются пары элементов из обоих матриц
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    # Разделение между вычислительными узлами происходит по строкам первой матрицы
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


# Возвращает произведения соответствующих элементов из обоих матриц, группируя их по индексу в результирующей матрице
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


# Суммирует полученные произведения
def REDUCE(key, values):
    (i, k) = key

    # solution code that yield(k3,v3) pairs
    k3 = (i, k)

    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# Соединение полученных произведений с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.25319977 1.18125354 0.90237901 1.00498342 0.46265433 1.03246214
  0.85955337 0.79462329 0.42846329 0.42826748 0.97167032 0.69732081
  0.9872058  1.20041221 0.77749769 0.78861158 0.41939602 1.04334869
  0.94644844 0.91874584 0.32665564 0.60187034 1.11852324 0.59997708
  0.71589833 0.46758557 0.9432099  0.46613929 0.90059563 0.34295497
  0.44466319 0.18543271 1.15526465 0.75666089 0.35220205 1.07797183
  0.79860139 0.50423639 0.73149463 0.43982244]
 [0.39368147 0.71047036 0.50191242 0.83473068 0.40578517 0.47723938
  0.48474465 0.68332965 0.32472158 0.52498063 0.44258011 0.76659093
  0.33973235 0.72439562 0.32665202 0.85501177 0.71610124 0.65041409
  0.7039553  0.3349843  0.49033622 0.54468901 0.77303424 0.38960818
  0.56273155 0.67348036 0.59867785 0.8028718  0.62493656 0.24857109
  0.59377953 0.2420048  0.66067822 0.46088426 0.45754182 0.5654878
  0.81445304 0.53323438 0.50740692 0.29002631]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Решение будет работать, так как каждая пара элементов, генерируемая RECORDREADER'ом, идентифицируется по ее индексам.